# Load project YAML from - Git, Zip, Tar source

After you create your project YAML you can simply load that project and run, build and deploy your function.
You can found an addional information of how to create a project YAML for CI/CD [here](https://github.com/GiladShapira94/load-project-example/blob/master/CI-CD%20automation%20creation.md)

In this session you would learn how to:
* Load or Create a new project for remote URL
* get a function object 
* run a function
* build a function
* deploy a function
* run a workflow

Install mlrun - if does not install use ``pip install mlrun==<mlrun server version>`` or ``sh align_mlrun.sh`` (our default mlrun installer - automatically install the server version)

In [1]:
import mlrun

In [2]:
source = 'git://github.com/GiladShapira94/quick-start.git#master'

#### Load or Create a new project
For loading a project you can use :
1. [load_project](https://docs.mlrun.org/en/latest/api/mlrun.projects.html?highlight=load_project#mlrun.projects.load_project) thats Load an MLRun project from git or tar or dir 
````
# load the project and run the 'main' workflow
project = load_project("./", "git://github.com/mlrun/project-demo.git",url=<name (in DB) or git or tar.gz or .zip sources archive path>)
````
2. [new_project](https://docs.mlrun.org/en/latest/api/mlrun.projects.html?highlight=new_project#mlrun.projects.new_project) thats Create a new MLRun project, optionally load it from a yaml/zip/git template.
````
# create a project with local and marketplace functions, a workflow, and an artifact
project = mlrun.new_project("myproj", "./", init_git=True, description="my new project",url=<name (in DB) or git or tar.gz or .zip sources archive path>)
````
***Important Note-*** If you want to run your code locally you must clone the files to the context directory, execute automaticlly when you load project or create new project and define a url params


In [3]:
project = mlrun.load_project('./clone',url=source,init_git=True,clone=True)

By Specifying a url/remote address MLRun automatically clone/extract your git repository ot tar/zip file

if Clone = True, always clone **(delete any existing content)**

In [4]:
print(project.to_yaml())

kind: project
metadata:
  name: quick-start-tutorial-shapira
spec:
  functions:
  - name: prep_data
    kind: job
    image: mlrun/mlrun
    handler: prep_data.prep_data
    with_repo: true
  - name: train_iris
    kind: job
    image: mlrun/mlrun
    handler: train.train_iris
    with_repo: true
  - url: serving.yaml
    name: serving
    with_repo: true
  workflows: []
  artifacts: []
  source: git://github.com/GiladShapira94/quick-start.git#refs/heads/master
  origin_url: git://github.com/GiladShapira94/quick-start.git#refs/heads/master
  load_source_on_run: true
  desired_state: online



#### Run Function 
For run a function you will need to use the [run_function](https://docs.mlrun.org/en/latest/api/mlrun.projects.html?highlight=run_function#mlrun.projects.MlrunProject.run_function) method.
This method allows you to run a MLRun **jobs** locally and remotely as long as there is no requirments ( if there is any requirments you will need to build a new image before you run a function)
It is equivalent to func.run method.
````
project.run_function(function='<function name>',params={'num':3},local=False)
````
````
project.run_function(function='<function name>',params={'num':3},local=True)
````

In [5]:
source_url = mlrun.get_sample_path("data/iris/iris.data.raw.csv")

In [6]:
project.run_function(function='prep_data',inputs={'source_url': source_url},local=True)

> 2022-09-19 13:28:34,966 [warning] run command, file or code were not specified
> 2022-09-19 13:28:34,968 [info] starting run prep_data-prep_data uid=dc1b6bb0036740b9b7a3ae812acb7b0a DB=http://mlrun-api:8080


project,uid,iter,start,state,name,labels,inputs,parameters,results,artifacts
quick-start-tutorial-shapira,...2acb7b0a,0,Sep 19 13:28:35,completed,prep_data-prep_data,v3io_user=shapirakind=localowner=shapirahost=jupyter-shapira-6d47768cf9-jqk9v,source_url,,num_rows=150,cleaned_data


> 2022-09-19 13:28:35,745 [info] run executed, status=completed


In [7]:
prep_data_run=project.run_function(function='prep_data',inputs={'source_url': source_url},local=False)

> 2022-09-19 13:28:37,864 [info] starting run prep_data-prep_data uid=a4cb45ee94e5469c8affce643816425d DB=http://mlrun-api:8080
> 2022-09-19 13:28:38,220 [info] Job is running in the background, pod: prep-data-prep-data-428zw
> 2022-09-19 13:28:41,908 [info] extracting source from git://github.com/GiladShapira94/quick-start.git#refs/heads/master to /mlrun/code
> 2022-09-19 13:28:42,287 [info] run executed, status=completed
final state: completed


project,uid,iter,start,state,name,labels,inputs,parameters,results,artifacts
quick-start-tutorial-shapira,...3816425d,0,Sep 19 13:28:41,completed,prep_data-prep_data,v3io_user=shapirakind=jobowner=shapiramlrun/client_version=1.1.0-rc22host=prep-data-prep-data-428zw,source_url,,num_rows=150,cleaned_data


> 2022-09-19 13:28:44,584 [info] run executed, status=completed


In [8]:
dataset = 'v3io:///projects/quick-start-tutorial-shapira/artifacts/prep_data-prep_data/0/cleaned_data.csv'

In [9]:
dataset

'v3io:///projects/quick-start-tutorial-shapira/artifacts/prep_data-prep_data/0/cleaned_data.csv'

In [10]:
train_run=project.run_function(function='train_iris',inputs={'dataset': dataset},params={'label_column': 'label'},local=False)

> 2022-09-19 13:28:44,835 [info] starting run train_iris-train_iris uid=38a3de229bca4d459320157e08d6bc96 DB=http://mlrun-api:8080
> 2022-09-19 13:28:45,094 [info] Job is running in the background, pod: train-iris-train-iris-6mtjz
> 2022-09-19 13:28:48,683 [info] extracting source from git://github.com/GiladShapira94/quick-start.git#refs/heads/master to /mlrun/code
> 2022-09-19 13:28:50,976 [info] run executed, status=completed
final state: completed


project,uid,iter,start,state,name,labels,inputs,parameters,results,artifacts
quick-start-tutorial-shapira,...08d6bc96,0,Sep 19 13:28:48,completed,train_iris-train_iris,v3io_user=shapirakind=jobowner=shapiramlrun/client_version=1.1.0-rc22host=train-iris-train-iris-6mtjz,dataset,label_column=label,accuracy=1.0f1_score=1.0precision_score=1.0recall_score=1.0auc-micro=1.0auc-macro=1.0auc-weighted=1.0,feature-importancetest_setconfusion-matrixroc-curvesmodel


> 2022-09-19 13:28:53,970 [info] run executed, status=completed


#### Get Function Object 
For gets function object you will need to use the [get_function](https://docs.mlrun.org/en/latest/api/mlrun.projects.html?highlight=get_function#mlrun.projects.MlrunProject.get_function) method.
This method allows you to get a function object based on the metadata in your project YAML file.
````
serving_func = project.get_function('<function name>')
````

In [29]:
serving_func = project.get_function('serving')

In [30]:
serving_func.add_model('model',model_path=project.get_artifact_uri('my_model'))
serving_func.spec.build.source=source

In [31]:
print(serving_func.to_yaml())

kind: serving
metadata:
  name: serving
  tag: ''
  hash: 8ca775c37943519dbe154ee18af2fb881c87836e
  project: quick-start-tutorial-shapira
  categories: []
spec:
  command: ''
  args: []
  image: mlrun/mlrun
  build:
    source: git://github.com/GiladShapira94/quick-start.git#master
    commands: []
    origin_filename: serving.py
  description: ''
  default_handler: ''
  disable_auto_mount: false
  volumes: []
  volume_mounts: []
  env: []
  resources:
    requests:
      memory: 1Mi
      cpu: 25m
    limits:
      memory: 20Gi
      cpu: '2'
  node_selector:
    app.iguazio.com/lifecycle: non-preemptible
  priority_class_name: igz-workload-medium
  preemption_mode: prevent
  min_replicas: 1
  max_replicas: 4
  base_spec:
    apiVersion: nuclio.io/v1
    kind: Function
    metadata:
      name: serving
      labels: {}
      annotations:
        nuclio.io/generated_by: function generated from serving.py
    spec:
      runtime: python
      handler: serving:handler
      env: []
    

Test your serving funciton locally before deploying it

In [33]:
my_data = '''{"inputs":[[5.1, 3.5, 1.4, 0.2],[7.7, 3.8, 6.7, 2.2]]}'''

In [34]:
serving_server = serving_func.to_mock_server()

> 2022-09-19 13:36:37,994 [info] model model was loaded
> 2022-09-19 13:36:37,995 [info] Loaded ['model']


In [35]:
serving_server.test('/',my_data)

X does not have valid feature names, but RandomForestClassifier was fitted with feature names


{'id': '8287fad7fab840b19abc293a6e94e3ec',
 'model_name': 'model',
 'outputs': [0, 2]}

**Changed pull_at_runtime to False for pull files on image creation**

In [36]:
project.set_source(source,pull_at_runtime=False)
print(project.to_yaml())

kind: project
metadata:
  name: quick-start-tutorial-shapira
spec:
  functions:
  - name: prep_data
    kind: job
    image: mlrun/mlrun
    handler: prep_data.prep_data
    with_repo: true
  - name: train_iris
    kind: job
    image: mlrun/mlrun
    handler: train.train_iris
    with_repo: true
  - url: serving.yaml
    name: serving
    with_repo: true
  workflows: []
  artifacts: []
  source: git://github.com/GiladShapira94/quick-start.git#master
  origin_url: git://github.com/GiladShapira94/quick-start.git#refs/heads/master
  load_source_on_run: false
  desired_state: online



#### Deploy Function
For deplying remote function as nuclio or serving you will need to use the [deploy_function](https://docs.mlrun.org/en/latest/api/mlrun.projects.html?highlight=deploy_function#mlrun.projects.MlrunProject.deploy_function) method.
You must use this method before invoke nuclio or serving fucntions.
It is equivalent to func.deploy() method.
````
nuclio_func=project.deploy_function(function='<function name>')

nuclio_func.function.invoke('/',{'int':4})
````

In [37]:
serving_func.deploy()

> 2022-09-19 13:36:42,038 [info] Starting remote function deploy
2022-09-19 13:36:45  (info) Deploying function
2022-09-19 13:36:45  (info) Building
2022-09-19 13:36:45  (info) Staging files and preparing base images
2022-09-19 13:36:45  (info) Building processor image
2022-09-19 13:38:00  (info) Build complete
2022-09-19 13:38:11  (info) Function deploy complete
> 2022-09-19 13:38:11,731 [info] successfully deployed function: {'internal_invocation_urls': ['nuclio-quick-start-tutorial-shapira-serving.default-tenant.svc.cluster.local:8080'], 'external_invocation_urls': ['quick-start-tutorial-shapira-serving-quick-start-tutor-i2w8f1na.default-tenant.app.cust-cs-il-3-4-1.iguazio-cd1.com/']}


'http://quick-start-tutorial-shapira-serving-quick-start-tutor-i2w8f1na.default-tenant.app.cust-cs-il-3-4-1.iguazio-cd1.com/'

In [40]:
serving_func.invoke('/',my_data)

> 2022-09-19 13:38:38,022 [info] invoking function: {'method': 'POST', 'path': 'http://nuclio-quick-start-tutorial-shapira-serving.default-tenant.svc.cluster.local:8080/'}


{'id': '3607a625-42e9-4ebd-8df8-380f7647f7a8',
 'model_name': 'model',
 'outputs': [0, 2]}